In [1]:
from PersistenceExplorer import *

# Computing persistence of images

We demonstrate the computation of persistence for a set of images. In the folder `../data/bmp` we have 20 images, as we see:

In [ ]:
[ filename for filename in os.listdir('../data/bmp') if filename.endswith('.bmp') ]

We can compute the sublevel and superlevel persistence of these images. The following command will compute the sublevel and superlevel persistence of each image and save the results as `.csv` files. Each image processed will result in a `.csv` file with the same basename, e.g. `00001.bmp` results in files  `pd_sub/00001.csv` and `pd_sup/00001.csv`.

In [ ]:
ProcessImageFolderWithPHAT('../data/bmp/')

We verify the sublevel results are indeed stored in a newly created subfolder `pd_sub`:

In [ ]:
[ filename for filename in os.listdir('../data/bmp/pd_sub') if filename.endswith('.csv') ]

Similarly the superlevel persistence results are stored in the subdirectory `pd_sup`. We can visually inspect the results of such a file:

In [ ]:
with open('../data/bmp/pd_sub/00001.csv', 'r') as f:
    csv_data = f.read()
print(csv_data)

### Finer control of persistence calculations

If finer control is required over which images to compute on and where to place the results, the user can directly call the following function (already available in the PersistenceExplorer package imported above):

```python
def ProcessImageListWithPHAT( list_of_image_filenames, list_of_output_filenames, filtration_type ):
  """
  Iterate through images, compute persistence results, and store results.
    list_of_image_filenames: a list of image files
    list_of_output_filenames: a list of files to save corresponding persistence results in
    filtration_type: either "sub" or "super" to indicate to obtain persistence results for either
                     sublevel or superlevel set filtrations
  """
  # Run commands in parallel
  processes = [subprocess.Popen(["ImagePersistence", infile, outfile, filtration_type]) for infile, outfile in zip(list_of_image_filenames, list_of_output_filenames) ]
  # Block until processing complete
  exitcodes = [p.wait() for p in processes]
```

# PersistenceExplorer

Reading the results of the persistence calculation as a long table of numbers is not in itself very illuminating. Instead, we can use a tool which allows us to visualize the results of the persistence calculation as a _persistence diagram_, and moreover, we can associate the persistence diagram points with features in the original image. The tool below provides the following abilities:

* We can load a sequence of images and their associated persistence results
* We can choose a range of frames of interest
* We choose a dimension of interested (i.e. $H_0$ features or $H_1$ persistence features).
* We provide the actual size of the image that the persistence diagram was generated from.
* We can choose a maximum size height/width for the displayed image (in pixels).
* We can choose a maximum size for the persistence diagram (in pixels).
* Selecting a rectangle in the image will highlight associated persistence generators in the persistence diagram
* Selecting a region in the persistence diagram with a lasso tool will display annotated features overlaid on the image and run an animation which shows how they evolve with time


In [17]:
imagefiles = [ '/files/data/bmp/' + filename for filename in os.listdir('../data/bmp') if filename.endswith('.bmp') ]
pdfiles = [ '/files/data/bmp/pd_sub/' + filename for filename in os.listdir('../data/bmp/pd_sub') if filename.endswith('.csv') ]
frames = range(0, len(imagefiles))
imagesize = [421, 421]
max_image_display_size = 400
persistence_diagram_display_size = 400

In [ ]:
dimension_of_interest = 0
PersistenceExplorer(imagefiles, pdfiles, frames, dimension_of_interest, 
                    imagesize, max_image_display_size, persistence_diagram_display_size)

In [ ]:
dimension_of_interest = 1
PersistenceExplorer(imagefiles, pdfiles, frames, dimension_of_interest,
                    imagesize, max_image_display_size, persistence_diagram_display_size)

## File path gotcha:

Note that above, the file paths used for `ProcessImageFolderWithPHAT` and for `PersistenceExplorer` are not the same.

`ProcessImageFolderWithPHAT` executes server side code which treats `/` as the root of the filesystem as recognized as the OS. 

`PersistenceExplorer`, on the other hand, is javascript which executes client-side. The client can only see the part of the filesystem which Jupyter serves over HTTP. The following example assumes `jupyter notebook` has been launched from the `tda-d3-explorer` directory. Jupyter Notebook treats this as the root directory, (which it names `/files` as opposed to just `/`). Thus we prefix all our paths with `/files` and can only access files underneath the file-system from wherever Jupyter Notebook was launched. For example `/path/to/tda-d3-explorer/data/bmp/00001.bmp` becomes `/files/data/bmp/00001.bmp` since Jupyter Notebook was launched in `/path/to/tda-d3-explorer`.

It is not clear yet the best way to resolve this confusion.